In [ ]:
import requests
import datetime
import pandas as pd
import os
from multiprocessing import Pool



In [11]:
import json

# List of cities you are interested in and their corresponding OpenWeatherMap city names
city_names = {
    'Anchorage': 'Anchorage',
    'Boise': 'Boise',
    'Chicago': 'Chicago',
    'Denver': 'Denver',
    'Detroit': 'Detroit',
    'Honolulu': 'Honolulu',
    'Houston': 'Houston',
    'Miami': 'Miami',
    'Minneapolis': 'Minneapolis',
    'Oklahoma City': 'Oklahoma City',
    'Nashville': 'Nashville',
    'New York': 'New York',
    'Phoenix': 'Phoenix',
    'Portland ME': 'Portland',
    'Portland OR': 'Portland',
    'Salt Lake City': 'Salt Lake City',
    'San Diego': 'San Diego',
    'San Francisco': 'San Francisco',
    'Seattle': 'Seattle',
    'Washington DC': 'Washington'
}

# Load the JSON data
with open('../data/openweather/current.city.list.json', 'r', encoding='utf-8') as f:
    city_data = json.load(f)

# Extract city information (adjusted for the JSON structure provided)
results = {}
for city in city_data:
    city_name = city['name']
    # Check if the city is in our list, and handle Portland distinction by country code and other attributes if available
    if city_name in city_names.values():
        if city_name == 'Portland':
            # Distinguish Portland ME from Portland OR (assuming US country and possible state data available)
            if city['country'] == 'US':
                # Assuming 'stat' key or any additional data to differentiate
                if (city_names['Portland ME'] == city_name and city.get('stat', {}).get('population', 0) < 100000) or \
                   (city_names['Portland OR'] == city_name and city.get('stat', {}).get('population', 0) >= 100000):
                    results[city['name']] = city.get('coord', {}).get('lat', 'N/A'), city.get('coord', {}).get('lon', 'N/A')
        else:
            results[city['name']] = city.get('coord', {}).get('lat', 'N/A'), city.get('coord', {}).get('lon', 'N/A')

# Print results
for city, coords in results.items():
    print(f"{city}: Latitude {coords[0]}, Longitude {coords[1]}")


San Francisco: Latitude 37.774929, Longitude -122.419418
Washington: Latitude 47.500118, Longitude -120.501472
San Diego: Latitude 32.715328, Longitude -117.157257
Miami: Latitude 25.774269, Longitude -80.193657
Oklahoma City: Latitude 35.46756, Longitude -97.516434
Nashville: Latitude 36.16589, Longitude -86.784439
Houston: Latitude 29.763281, Longitude -95.363274
Portland: Latitude 45.523449, Longitude -122.676208
Chicago: Latitude 41.850029, Longitude -87.650047
Detroit: Latitude 42.331429, Longitude -83.045753
Minneapolis: Latitude 44.979969, Longitude -93.26384
New York: Latitude 43.000351, Longitude -75.499901
Phoenix: Latitude 33.44838, Longitude -112.074043
Denver: Latitude 39.739151, Longitude -104.984703
Boise: Latitude 43.613499, Longitude -116.203453
Salt Lake City: Latitude 40.76078, Longitude -111.891052
Seattle: Latitude 47.606209, Longitude -122.332069
Honolulu: Latitude 21.30694, Longitude -157.858337
Anchorage: Latitude 61.21806, Longitude -149.900284


In [1]:
import requests

# Your OpenWeatherMap API key
API_KEY = '59016685e15f5d93325917ec6e2e1fb1'  # Replace with your actual API key

# Example latitude and longitude
lat = 36.16589
lon = -86.784439

# Base URL for OpenWeatherMap's Current Weather API
url = "https://api.openweathermap.org/data/2.5/weather?lat=36.16&lon=-86.78&appid=d817ee55d93f6b51d9b15bc850821726&units=imperial"

# Make the API request
response = requests.get(url)

# Check response status
if response.status_code == 200:
    weather_data = response.json()
    print("Current Weather Data:")
    print(weather_data)
else:
    print(f"Error: Unable to fetch data. Status code: {response.status_code}")


/Users/judydw/Documents/UMich/Fall 2024/STATS 604/Project1/pythonProject/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Current Weather Data:
{'coord': {'lon': -86.78, 'lat': 36.16}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 55.24, 'feels_like': 54.03, 'temp_min': 54.09, 'temp_max': 56.26, 'pressure': 1017, 'humidity': 76, 'sea_level': 1017, 'grnd_level': 995}, 'visibility': 10000, 'wind': {'speed': 4.61, 'deg': 300}, 'clouds': {'all': 100}, 'dt': 1731637666, 'sys': {'type': 1, 'id': 4609, 'country': 'US', 'sunrise': 1731587014, 'sunset': 1731623996}, 'timezone': -21600, 'id': 4617305, 'name': 'Davidson', 'cod': 200}


In [34]:
import requests
import csv
import os

# Your OpenWeatherMap API key
API_KEY = 'd817ee55d93f6b51d9b15bc850821726'  # Replace with your actual API key

# Dictionary containing city names and their coordinates (latitude, longitude)
city_coordinates = {
    'Anchorage': (61.2181, -149.9003),
    'Boise': (43.615, -116.2023),
    'Chicago': (41.8781, -87.6298),
    'Denver': (39.7392, -104.9903),
    'Detroit': (42.3314, -83.0458),
    'Honolulu': (21.3069, -157.8583),
    'Houston': (29.7604, -95.3698),
    'Miami': (25.7617, -80.1918),
    'Minneapolis': (44.9778, -93.265),
    'Oklahoma City': (35.4676, -97.5164),
    'Nashville': (36.1627, -86.7816),
    'New York': (40.7128, -74.006),
    'Phoenix': (33.4484, -112.074),
    'Portland ME': (43.6591, -70.2568),
    'Portland OR': (45.5051, -122.675),
    'Salt Lake City': (40.7608, -111.891),
    'San Diego': (32.7157, -117.1611),
    'San Francisco': (37.7749, -122.4194),
    'Seattle': (47.6062, -122.3321),
    'Washington DC': (38.9072, -77.0369)
}

BASE_URL = 'https://api.openweathermap.org/data/2.5/weather'

# Directory to save the combined weather data file
output_dir = '../data/openweather/current'
os.makedirs(output_dir, exist_ok=True)

# CSV file path
csv_file_path = os.path.join(output_dir, "detailed_weather_data.csv")

# Fetch weather data and save to CSV
with open(csv_file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write the header row
    writer.writerow([
        "City", "Latitude", "Longitude", "Weather Description", "Temperature (F)", "Feels Like (F)",
        "Temp Min (F)", "Temp Max (F)", "Pressure", "Humidity", "Sea Level", "Ground Level",
        "Wind Speed", "Wind Deg", "Wind Gust", "Clouds All", "Datetime", "Country",
        "Sunrise", "Sunset", "Timezone", "City ID", "City Name"
    ])

    # Function to fetch weather data for a city
    def fetch_weather(city, lat, lon):
        params = {
            'lat': lat,
            'lon': lon,
            'appid': API_KEY,
            'units': 'imperial'  # Use 'metric' for Celsius
        }
        response = requests.get(BASE_URL, params=params)
        if response.status_code == 200:
            weather_data = response.json()
            # Extract relevant data
            coord = weather_data.get('coord', {})
            weather = weather_data.get('weather', [{}])[0]
            main = weather_data.get('main', {})
            wind = weather_data.get('wind', {})
            clouds = weather_data.get('clouds', {})
            sys = weather_data.get('sys', {})

            # Create a row with all requested data, handling missing values
            return [
                city,
                coord.get('lat', 'N/A'), coord.get('lon', 'N/A'),
                weather.get('description', 'N/A'),
                main.get('temp', 'N/A'), main.get('feels_like', 'N/A'),
                main.get('temp_min', 'N/A'), main.get('temp_max', 'N/A'),
                main.get('pressure', 'N/A'), main.get('humidity', 'N/A'),
                main.get('sea_level', 'N/A'), main.get('grnd_level', 'N/A'),
                wind.get('speed', 'N/A'), wind.get('deg', 'N/A'), wind.get('gust', 'N/A'),
                clouds.get('all', 'N/A'),
                weather_data.get('dt', 'N/A'), sys.get('country', 'N/A'),
                sys.get('sunrise', 'N/A'), sys.get('sunset', 'N/A'),
                weather_data.get('timezone', 'N/A'), weather_data.get('id', 'N/A'),
                weather_data.get('name', 'N/A')
            ]
        else:
            print(f"Error fetching data for {city}. Status code: {response.status_code}")
            return [city] + ["Error"] * 21

    # Fetch and write weather data for each city
    for city, (lat, lon) in city_coordinates.items():
        city_weather = fetch_weather(city, lat, lon)
        writer.writerow(city_weather)

print(f"Detailed weather data saved to {csv_file_path}")

Detailed weather data saved to ../data/openweather/current/detailed_weather_data.csv


In [2]:
 import pandas as pd
 combined = pd.read_csv("../data/openweather/current/detailed_weather_data.csv")
 combined.head(10)

,City,Latitude,Longitude,Weather Description,Temperature (F),Feels Like (F),Temp Min (F),Temp Max (F),Pressure,Humidity,...,Wind Deg,Wind Gust,Clouds All,Datetime,Country,Sunrise,Sunset,Timezone,City ID,City Name
0,Anchorage,61.2181,-149.9003,mist,15.62,6.98,11.57,19.09,1007,88,...,80,NaN,0,1731551069,US,1731520923,1731547568,-32400,5879400,Anchorage
1,Boise,43.6150,-116.2023,overcast clouds,51.03,47.95,46.17,52.52,1011,45,...,220,NaN,100,1731551067,US,1731508624,1731543690,-25200,5586437,Boise
2,Chicago,41.8781,-87.6298,light rain,48.24,41.09,46.53,49.08,1012,91,...,129,28.01,100,1731550970,US,1731501504,1731537093,-21600,4887398,Chicago
3,Denver,39.7420,-104.9915,overcast clouds,38.61,38.61,36.10,41.22,1017,65,...,285,0.74,100,1731550809,US,1731505380,1731541553,-25200,5419384,Denver
4,Detroit,42.3316,-83.0466,overcast clouds,50.07,47.88,48.25,52.07,1019,66,...,140,NaN,100,1731551182,US,1731500469,1731535929,-18000,4990729,Detroit
5,Honolulu,21.3133,-157.8578,shower rain,80.82,83.39,77.92,82.26,1018,64,...,50,31.07,40,1731550699,US,1731516096,1731556215,-36000,5856195,Honolulu
6,Houston,29.7604,-95.3698,scattered clouds,74.30,74.34,70.95,76.14,1016,62,...,311,13.00,40,1731550834,US,1731501903,1731540410,-21600,4699066,Houston
7,Miami,25.7629,-80.1895,scattered clouds,77.59,78.44,76.01,79.02,1017,72,...,80,25.32,40,1731551003,US,1731497860,1731537166,-18000,4164138,Miami
8,Minneapolis,44.9773,-93.2655,mist,42.89,41.16,41.41,44.56,1013,95,...,140,NaN,100,1731550777,US,1731503326,1731537977,-21600,5037649,Minneapolis
9,Oklahoma City,35.4676,-97.5164,clear sky,53.42,51.28,49.89,55.62,1019,60,...,290,NaN,0,1731550607,US,1731503048,1731540296,-21600,4544349,Oklahoma City


In [3]:
combined.columns

Index(['City', 'Latitude', 'Longitude', 'Weather Description',
       'Temperature (F)', 'Feels Like (F)', 'Temp Min (F)', 'Temp Max (F)',
       'Pressure', 'Humidity', 'Sea Level', 'Ground Level', 'Wind Speed',
       'Wind Deg', 'Wind Gust', 'Clouds All', 'Datetime', 'Country', 'Sunrise',
       'Sunset', 'Timezone', 'City ID', 'City Name'],
      dtype='object')

In [20]:
import requests
import time
import json  # Import the json module

# Your OpenWeatherMap API Key
API_KEY = 'd817ee55d93f6b51d9b15bc850821726'
# Coordinates for Detroit (or any other location)
lat = 42.3314
lon = -83.0458

# Define the time range: past 1 year (in Unix timestamps)
end_time = int(time.time())  # Current time in Unix timestamp
start_time = end_time - (365 * 24 * 3600)  # One year ago in Unix timestamp

# Construct the URL for the API call with units=imperial for Fahrenheit
url = f'https://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&start={start_time}&end={end_time}&units=imperial&appid={API_KEY}'

# Send the API request
response = requests.get(url)
data_json = response.json()

# Check if the request was successful
if response.status_code == 200 and 'list' in data_json:
    print("Data retrieval successful!")
    # Save the data as a JSON file for later use
    with open('detroit_hourly_data_fahrenheit.json', 'w') as json_file:
        json.dump(data_json, json_file, indent=4)
    print("Data saved to 'detroit_hourly_data_fahrenheit.json'.")
else:
    print(f"Failed to retrieve data. Response: {data_json}")

Data retrieval successful!
Data saved to 'detroit_hourly_data_fahrenheit.json'.


In [35]:
data_json

{'message': 'Count: 168',
 'cod': '200',
 'city_id': 1,
 'calctime': 0.14805059,
 'cnt': 168,
 'list': [{'dt': 1700190000,
   'main': {'temp': 50.72,
    'feels_like': 48.45,
    'pressure': 1021,
    'humidity': 63,
    'temp_min': 44.74,
    'temp_max': 56.48},
   'wind': {'speed': 0, 'deg': 0},
   'clouds': {'all': 0},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01n'}]},
  {'dt': 1700193600,
   'main': {'temp': 50.2,
    'feels_like': 48.16,
    'pressure': 1021,
    'humidity': 69,
    'temp_min': 42.94,
    'temp_max': 55.47},
   'wind': {'speed': 8.05, 'deg': 170},
   'clouds': {'all': 0},
   'weather': [{'id': 800,
     'main': 'Clear',
     'description': 'clear sky',
     'icon': '01n'}]},
  {'dt': 1700197200,
   'main': {'temp': 50.34,
    'feels_like': 48.51,
    'pressure': 1021,
    'humidity': 73,
    'temp_min': 42.96,
    'temp_max': 55.38},
   'wind': {'speed': 6.91, 'deg': 180},
   'clouds': {'all': 0},
   'weather':

In [15]:
import json
import pandas as pd
from datetime import datetime

# Load the Detroit hourly data from the JSON file
data = data_json

# Initialize an empty list to store the rows of data
# Initialize an empty list to store the rows of data
rows = []

# Loop through each entry in the "list" field of the JSON data
for entry in data['list']:
    # Extract the Unix timestamp and convert to datetime
    dt = datetime.utcfromtimestamp(entry['dt']).strftime('%Y-%m-%d %H:%M:%S')
    
    # Extract temperature, which is already in Fahrenheit
    temp = entry['main']['temp']
    feels_like = entry['main']['feels_like']
    temp_min = entry['main']['temp_min']
    temp_max = entry['main']['temp_max']
    
    # Extract pressure and humidity
    pressure = entry['main']['pressure']
    humidity = entry['main']['humidity']
    
    # Extract wind details
    wind_speed = entry['wind']['speed']
    wind_deg = entry['wind']['deg']
    wind_gust = entry['wind'].get('gust', None)
    
    # Extract cloud coverage
    clouds_all = entry['clouds']['all']
    
    # Extract weather description
    weather_description = entry['weather'][0]['description']
    
    # Append the data as a row
    rows.append([dt, temp, feels_like, temp_min, temp_max, pressure, humidity, wind_speed, wind_deg, 
                 wind_gust, clouds_all, weather_description])

# Create a DataFrame with the specified column names
df = pd.DataFrame(rows, columns=['Datetime', 'Temperature (F)', 'Feels Like (F)', 'Temp Min (F)', 'Temp Max (F)',
                                 'Pressure', 'Humidity', 'Wind Speed', 'Wind Deg', 'Wind Gust', 'Clouds All',
                                 'Weather Description'])

# Save the DataFrame to a CSV file
df.to_csv('Detroit_hourly_data.csv', index=False)

print("Data saved to 'Detroit_hourly_data.csv'.")

Data saved to 'Detroit_hourly_data.csv'.


In [21]:
import requests
import time
import json
import os  # Import os to manage file paths

# Your OpenWeatherMap API Key
API_KEY = 'd817ee55d93f6b51d9b15bc850821726'

# Coordinates for each location
locations = {
    "Anchorage": {"lat": 61.2181, "lon": -149.9003, "id": "PANC"},
    "Boise": {"lat": 43.6150, "lon": -116.2023, "id": "KBOI"},
    "Chicago": {"lat": 41.8781, "lon": -87.6298, "id": "KORD"},
    "Denver": {"lat": 39.7392, "lon": -104.9903, "id": "KDEN"},
    "Detroit": {"lat": 42.3314, "lon": -83.0458, "id": "KDTW"},
    "Honolulu": {"lat": 21.3069, "lon": -157.8583, "id": "PHNL"},
    "Houston": {"lat": 29.7604, "lon": -95.3698, "id": "KIAH"},
    "Miami": {"lat": 25.7617, "lon": -80.1918, "id": "KMIA"},
    "Minneapolis": {"lat": 44.9778, "lon": -93.2650, "id": "KMSP"},
    "Oklahoma City": {"lat": 35.4676, "lon": -97.5164, "id": "KOKC"},
    "Nashville": {"lat": 36.1627, "lon": -86.7816, "id": "KBNA"},
    "New York": {"lat": 40.7128, "lon": -74.0060, "id": "KJFK"},
    "Phoenix": {"lat": 33.4484, "lon": -112.0740, "id": "KPHX"},
    "Portland ME": {"lat": 43.6591, "lon": -70.2568, "id": "KPWM"},
    "Portland OR": {"lat": 45.5051, "lon": -122.6750, "id": "KPDX"},
    "Salt Lake City": {"lat": 40.7608, "lon": -111.8910, "id": "KSLC"},
    "San Diego": {"lat": 32.7157, "lon": -117.1611, "id": "KSAN"},
    "San Francisco": {"lat": 37.7749, "lon": -122.4194, "id": "KSFO"},
    "Seattle": {"lat": 47.6062, "lon": -122.3321, "id": "KSEA"},
    "Washington DC": {"lat": 38.9072, "lon": -77.0369, "id": "KDCA"}
}

# Define the time range: past 1 year (in Unix timestamps)
end_time = int(time.time())  # Current time in Unix timestamp
start_time = end_time - (364 * 24 * 3600)  # One year ago in Unix timestamp

# Directory where data will be saved
directory = '../data/original/openweather_hourly/'

# Ensure the directory exists
os.makedirs(directory, exist_ok=True)

# Loop through each location and retrieve the historical weather data
for city, coords in locations.items():
    lat = coords["lat"]
    lon = coords["lon"]
    
    # Construct the URL for the API call with units=imperial for Fahrenheit
    url = f'https://history.openweathermap.org/data/2.5/history/city?lat={lat}&lon={lon}&type=hour&start={start_time}&end={end_time}&units=imperial&appid={API_KEY}'

    # Send the API request
    response = requests.get(url)
    data_json = response.json()

    # Check if the request was successful
    if response.status_code == 200 and 'list' in data_json:
        print(f"Data retrieval successful for {city}!")
        # Save the data as a JSON file for later use
        file_name = f'{city}_hourly_data.json'
        file_path = os.path.join(directory, file_name)
        
        with open(file_path, 'w') as json_file:
            json.dump(data_json, json_file, indent=4)
        print(f"Data saved to '{file_path}'.")
    else:
        print(f"Failed to retrieve data for {city}. Response: {data_json}")


Data retrieval successful for Anchorage!
Data saved to '../data/original/openweather_hourly/Anchorage_hourly_data.json'.
Data retrieval successful for Boise!
Data saved to '../data/original/openweather_hourly/Boise_hourly_data.json'.
Data retrieval successful for Chicago!
Data saved to '../data/original/openweather_hourly/Chicago_hourly_data.json'.
Data retrieval successful for Denver!
Data saved to '../data/original/openweather_hourly/Denver_hourly_data.json'.
Data retrieval successful for Detroit!
Data saved to '../data/original/openweather_hourly/Detroit_hourly_data.json'.
Data retrieval successful for Honolulu!
Data saved to '../data/original/openweather_hourly/Honolulu_hourly_data.json'.
Data retrieval successful for Houston!
Data saved to '../data/original/openweather_hourly/Houston_hourly_data.json'.
Data retrieval successful for Miami!
Data saved to '../data/original/openweather_hourly/Miami_hourly_data.json'.
Data retrieval successful for Minneapolis!
Data saved to '../data/or

In [36]:
import json
import os
import pandas as pd
from datetime import datetime

# Function to process and save data to CSV
def process_and_save_data(data_json, city_name):
    # Initialize an empty list to store the rows of data
    rows = []

    # Loop through each entry in the "list" field of the JSON data
    for entry in data_json['list']:
        # Extract the Unix timestamp and convert to datetime
        dt = datetime.utcfromtimestamp(entry['dt'])
        
        # Extract the components: year, month, day, time
        year = dt.year
        month = dt.month
        day = dt.day
        time = dt.strftime('%H:%M:%S')  # Time in HH:MM:SS format

        # Extract temperature, which is already in Fahrenheit
        temp = entry['main']['temp']
        feels_like = entry['main']['feels_like']
        temp_min = entry['main']['temp_min']
        temp_max = entry['main']['temp_max']

        # Extract pressure and humidity
        pressure = entry['main']['pressure']
        humidity = entry['main']['humidity']

        # Extract wind details
        wind_speed = entry['wind']['speed']
        wind_deg = entry['wind']['deg']
        wind_gust = entry['wind'].get('gust', None)

        # Extract cloud coverage
        clouds_all = entry['clouds']['all']

        # Extract weather description
        weather_description = entry['weather'][0]['description']

        # Append the data as a row
        rows.append([year, month, day, time, temp, feels_like, temp_min, temp_max, pressure, humidity, wind_speed,
                     wind_deg, wind_gust, clouds_all, weather_description])

    # Create a DataFrame with the specified column names
    df = pd.DataFrame(rows, columns=['Year', 'Month', 'Day', 'Time', 'Temperature (F)', 'Feels Like (F)', 
                                     'Temp Min (F)', 'Temp Max (F)', 'Pressure', 'Humidity', 'Wind Speed', 
                                     'Wind Deg', 'Wind Gust', 'Clouds All', 'Weather Description'])

    # Ensure the directory exists
    directory = '../data/processed/openweather_hourly'
    os.makedirs(directory, exist_ok=True)

    # Define the file path for saving the CSV
    file_path = os.path.join(directory, f'{city_name}_hourly_data.csv')

    # Save the DataFrame to a CSV file
    df.to_csv(file_path, index=False)
    print(f"Data saved to '{file_path}'.")

# Function to process all JSON files and convert them to CSV
def process_all_json_files(json_directory):
    # Get the list of all JSON files in the given directory
    json_files = [f for f in os.listdir(json_directory) if f.endswith('_hourly_data.json')]

    # Process each JSON file
    for json_file in json_files:
        city_name = json_file.split('_')[0]  # Extract city name from the filename (assuming format 'city_name_hourly_data_fahrenheit.json')
        json_path = os.path.join(json_directory, json_file)
        
        # Load the JSON data
        with open(json_path, 'r') as file:
            data_json = json.load(file)
        
        # Process and save the data as CSV
        process_and_save_data(data_json, city_name)

# Directory where the JSON files are stored
json_directory = '../data/original/openweather_hourly'

# Call the function to process all JSON files in the specified directory
process_all_json_files(json_directory)


Data saved to '../data/processed/openweather_hourly/New York_hourly_data.csv'.
Data saved to '../data/processed/openweather_hourly/Chicago_hourly_data.csv'.
Data saved to '../data/processed/openweather_hourly/Nashville_hourly_data.csv'.
Data saved to '../data/processed/openweather_hourly/Portland ME_hourly_data.csv'.
Data saved to '../data/processed/openweather_hourly/Salt Lake City_hourly_data.csv'.
Data saved to '../data/processed/openweather_hourly/Oklahoma City_hourly_data.csv'.
Data saved to '../data/processed/openweather_hourly/San Diego_hourly_data.csv'.
Data saved to '../data/processed/openweather_hourly/Washington DC_hourly_data.csv'.
Data saved to '../data/processed/openweather_hourly/Houston_hourly_data.csv'.
Data saved to '../data/processed/openweather_hourly/Boise_hourly_data.csv'.
Data saved to '../data/processed/openweather_hourly/Anchorage_hourly_data.csv'.
Data saved to '../data/processed/openweather_hourly/San Francisco_hourly_data.csv'.
Data saved to '../data/process

In [37]:
dc = pd.read_csv('../data/processed/openweather_hourly/Washington DC_hourly_data.csv')
dc.head(10)

,Year,Month,Day,Time,Temperature (F),Feels Like (F),Temp Min (F),Temp Max (F),Pressure,Humidity,Wind Speed,Wind Deg,Wind Gust,Clouds All,Weather Description
0,2023,11,17,03:00:00,50.72,48.45,44.74,56.48,1021,63,0.00,0,NaN,0,clear sky
1,2023,11,17,04:00:00,50.20,48.16,42.94,55.47,1021,69,8.05,170,NaN,0,clear sky
2,2023,11,17,05:00:00,50.34,48.51,42.96,55.38,1021,73,6.91,180,NaN,0,clear sky
3,2023,11,17,06:00:00,49.68,45.91,42.73,54.48,1020,76,9.22,180,NaN,20,few clouds
4,2023,11,17,07:00:00,49.24,47.32,42.24,54.25,1020,79,5.03,194,20.76,3,clear sky
5,2023,11,17,08:00:00,48.79,45.77,42.24,53.08,1019,82,6.91,190,NaN,0,clear sky
6,2023,11,17,09:00:00,48.69,46.22,43.21,53.24,1018,78,5.77,188,23.31,6,clear sky
7,2023,11,17,10:00:00,48.42,45.73,43.81,53.24,1019,85,6.08,188,24.81,5,clear sky
8,2023,11,17,11:00:00,47.68,45.34,42.24,52.25,1018,86,5.23,195,22.44,24,few clouds
9,2023,11,17,12:00:00,47.32,44.92,43.25,53.47,1019,91,5.23,192,21.36,35,scattered clouds
